In [ ]:
!pip install pandas 
!pip install dash-bootstrap-components
!pip install dash


Importação de Bibliotecas e Definição de Funções:

Carregamento e Pré-processamento dos Dados:

Definição do Layout do Dashboard:

Callbacks para Interatividade dos Gráficos:

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

# Carregar os dados
tabela = pd.read_csv("acidente-transito.csv", sep=';', encoding='latin-1')

# Preprocessamento dos dados
tabela.columns = tabela.columns.str.strip()
tabela = tabela.dropna()
tabela = tabela.drop(columns=["Nº_boletim", "seq_veic"])
tabela['data_hora_boletim'] = pd.to_datetime(tabela['data_hora_boletim'], format='%d/%m/%Y %H:%M')
tabela['hora'] = tabela['data_hora_boletim'].dt.hour
tabela['mes'] = tabela['data_hora_boletim'].dt.month

# Iniciar o aplicativo Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout do aplicativo
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Análise de Acidentes de Trânsito", className="text-center text-primary mb-4"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Selecione a Hora:"),
            dcc.Slider(
                id='hora-slider',
                min=0,
                max=23,
                step=1,
                marks={i: f'{i}h' for i in range(24)},
                value=12,
                className="mb-4"
            )
        ], width=12)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='grafico-tipos-socorro'), width=6),
        dbc.Col(dcc.Graph(id='grafico-veiculos-envolvidos'), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='grafico-situacoes-veiculo'), width=6),
        dbc.Col(dcc.Graph(id='grafico-distribuicao-veiculos'), width=6)
    ])
], fluid=True)

# Callback para atualizar os gráficos com base na hora selecionada
@app.callback(
    [Output('grafico-tipos-socorro', 'figure'),
     Output('grafico-veiculos-envolvidos', 'figure'),
     Output('grafico-situacoes-veiculo', 'figure'),
     Output('grafico-distribuicao-veiculos', 'figure')],
    [Input('hora-slider', 'value')]
)
def update_graphs(hora):
    # Filtrar os dados com base na hora selecionada
    tabela_filtrada = tabela[tabela['hora'] == hora]
    
    # Gráfico de Tipos de Socorro
    tipos_socorro = tabela_filtrada['desc_tipo_socorro'].value_counts()
    fig_socorro = px.bar(tipos_socorro, x=tipos_socorro.index, y=tipos_socorro.values, 
                         labels={'x': 'Tipos de Socorro', 'y': 'Quantidade'}, title='Tipos de Socorro',
                         color_discrete_sequence=['#636EFA'])

    # Gráfico de Veículos Envolvidos
    veiculos = tabela_filtrada['descricao_categoria'].value_counts()
    fig_veiculos = px.bar(veiculos, x=veiculos.index, y=veiculos.values, 
                          labels={'x': 'Veículos Envolvidos', 'y': 'Quantidade'}, title='Veículos Envolvidos',
                          color_discrete_sequence=['#EF553B'])

    # Gráfico de Situações do Veículo
    situacoes_veiculo = tabela_filtrada['desc_situacao'].value_counts()
    fig_situacoes = px.bar(situacoes_veiculo, x=situacoes_veiculo.index, y=situacoes_veiculo.values, 
                           labels={'x': 'Situação do Veículo', 'y': 'Quantidade'}, title='Situação do Veículo',
                           color_discrete_sequence=['#00CC96'])
    
    # Gráfico de Distribuição de Veículos (Pizza)
    especie_count = tabela_filtrada['descricao_especie'].value_counts()
    especie_perc = 100. * especie_count / especie_count.sum()
    especie_filtrado = especie_perc[especie_perc >= 1.7]
    fig_distribuicao = px.pie(values=especie_filtrado.values, names=especie_filtrado.index, 
                              title='Distribuição de Veículos (acima de 1.7%)',
                              color_discrete_sequence=px.colors.sequential.RdBu)
    
    return fig_socorro, fig_veiculos, fig_situacoes, fig_distribuicao

# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)
